# `nndex` Python Bindings Demo

Demonstrates the current Python API with `NNdex` for exact/ANN, batch search, and optional backend selection.


In [1]:
import time
import numpy as np

from nndex import NNdex

In [2]:
rows, dims = 50_000, 128
rng = np.random.default_rng(42)
matrix = rng.normal(size=(rows, dims)).astype(np.float32)
query = rng.normal(size=(dims,)).astype(np.float32)
queries = rng.normal(size=(4, dims)).astype(np.float32)


In [3]:
cpu_exact = NNdex(matrix, normalized=False, approx=False, backend="cpu")
cpu_ann = NNdex(matrix, normalized=False, approx=True, backend="cpu")
auto_backend = NNdex(matrix, normalized=False, approx=False, backend="auto")

print("cpu_exact backend:", cpu_exact.backend)
print("cpu_ann backend:", cpu_ann.backend)
print("auto backend:", auto_backend.backend)


cpu_exact backend: cpu
cpu_ann backend: cpu
auto backend: gpu


In [4]:
def time_call(model: NNdex, q: np.ndarray, k: int = 5, repeats: int = 25) -> float:
    timings_us = []
    for _ in range(repeats):
        t0 = time.perf_counter_ns()
        model.search(q, k=k)
        timings_us.append((time.perf_counter_ns() - t0) / 1_000.0)
    return float(np.median(timings_us))

print(f"CPU exact median single-query latency: {time_call(cpu_exact, query):.2f} us")
print(f"CPU ANN median single-query latency:   {time_call(cpu_ann, query):.2f} us")


CPU exact median single-query latency: 289.62 us
CPU ANN median single-query latency:   1.17 us


In [5]:
indices, scores = cpu_exact.search(query, k=5)
print("Single-query top-5 indices:", indices.tolist())
print("Single-query top-5 scores:", scores.tolist())

batch_indices, batch_scores = cpu_exact.search(queries, k=5)
print("Batch index shape:", batch_indices.shape)
print("Batch score shape:", batch_scores.shape)


Single-query top-5 indices: [43576, 14100, 15993, 35409, 38916]
Single-query top-5 scores: [0.36015188694000244, 0.35316410660743713, 0.33494800329208374, 0.33226925134658813, 0.323153018951416]
Batch index shape: (4, 5)
Batch score shape: (4, 5)
